In [ ]:
1. modify column name X.rename()
2. patsy.dmatrices ...dummy variable
3. crosstab, hist diagram

In [ ]:
import numpy as np # 数组常用库
import pandas as pd # 读入csv常用库
from patsy import dmatrices # 可根据离散变量自动生成哑变量
from sklearn.linear_model import LogisticRegression # sk-learn库Logistic Regression模型
from sklearn.model_selection import train_test_split, cross_val_score # sk-learn库训练与测试
from sklearn import metrics # 生成各项测试指标库
import matplotlib.pyplot as plt # 画图常用库

**> 从../input/HR_comma_sep.csv文件中读入数据，存入data

In [ ]:
data = pd.read_csv("../input/HR_comma_sep.csv")
data

In [ ]:
data.dtypes
data.left = data.left.astype(int)

In [ ]:
pd.crosstab(data.salary, data.left).plot(kind="bar")

In [ ]:
pd.crosstab(data.salary, data.left)

In [ ]:
q = pd.crosstab(data.salary, data.left)
print(q)
print("\n")
print(q.sum(1))
print("\n")
print(q.div(q.sum(1), axis=0))
q.div(q.sum(1), axis=0).plot(kind = "bar", stacked="true")



观察离职人数与工资分布的关系

In [ ]:
left_low = data['left'][data['salary'] == 'low'].value_counts()
left_medium = data['left'][data['salary'] == 'medium'].value_counts()
left_high = data['left'][data['salary'] == 'high'].value_counts()

df = pd.DataFrame({"low":left_low, "medium":left_medium, "high": left_high})
print(df)
df.plot(kind = "bar", stacked="true")
# 查看某离散变量里 各种可能的取值与 label的关系
pd.crosstab(data.left, data.salary).plot(kind="bar")
plt.show()

观察离职比例与工资分布的关系

In [ ]:
q = pd.crosstab(data.salary, data.left)
print(q)
print("\n")
#for each row, add all columm's value
print(q.sum(1))
print("\n")
#for each column, add all row's value
print(q.sum(0))

q.div(q.sum(1), axis=0).plot(kind="bar", stacked="true")

print("\n")
q1 = pd.crosstab(data.left, data.salary)
print(q1)
q1.div(q1.sum(1), axis=0).plot(kind="bar", stacked="true")

In [ ]:
观察员工满意度的分布图(histogram)

In [ ]:
# have to use plt.show() here, otherwise, those two diagram will be merged together
data.satisfaction_level[data.left == 1].hist()
plt.show()

data.satisfaction_level[data.left == 0].hist()
plt.show()


dmatrices将数据中的离散变量变成哑变量，并指明用satisfaction_level, last_evaluation, ... 来预测left

In [ ]:
model = LogisticRegression()
# left is the label, once, dmatrices is done, a new column named  "Intercept" is generated
# Intercept will digest one column from dummy variable i.e. low, high, medium -> low medium which means high is digested by Intercept
# left 就是label, 待预测的变量
# sales(account, sales, 部门)， salary(low, medium high)： 这两个变量本身就是离散变量，由于通过dmatrices 产生的新的列名称 比较fancy所以需要下面re-name
# C(sales) 相当于告诉程序 这是离散变量，请帮我变成dummy variable
y, X = dmatrices('left~satisfaction_level+last_evaluation+number_project+average_montly_hours+time_spend_company+Work_accident+promotion_last_5years+C(sales)+C(salary)', data, return_type = 'dataframe')
print(type(X))
print(type(y))
X.head()


In [ ]:
# those fancy columns names are generated by above dmatrices function, have to rename them
X = X.rename(columns = {
    'C(sales)[T.RandD]': 'Department: Random',
    'C(sales)[T.accounting]': 'Department: Accounting',
    'C(sales)[T.hr]': 'Department: HR',
    'C(sales)[T.management]': 'Department: Management',
    'C(sales)[T.marketing]': 'Department: Marketing',
    'C(sales)[T.product_mng]': 'Department: Product_Management',
    'C(sales)[T.sales]': 'Department: Sales',
    'C(sales)[T.support]': 'Department: Support',
    'C(sales)[T.technical]': 'Department: Technical',
    'C(salary)[T.low]': 'Salary: Low',
    'C(salary)[T.medium]': 'Salary: Medium'}) 
y = np.ravel(y) # 将y变成np的一维数组
X.head()

用X和y训练模型，然后输出X中每一项自变量对于y的影响
zip(a,b)可将a的每一个元素和b里对应位置的元素组成一对

In [ ]:
model.fit(X, y)
pd.DataFrame(list(zip(X.columns, np.transpose(model.coef_))))


model.score为准确率(0到1之间)

In [ ]:
print(model.score(X,y))
model.coef_

预测这样一个人的离职概率：
一个高工资HR，
对公司满意度0.5, 
上次评审0.7分，
做过4个项目，
每月平均工作160小时，
在公司呆了3年，
过去5年没有被晋升，
没有工伤


下面model.predict_proba会 输出：  array([[0.91615561, 0.08384439]]) 0.08384439 即表示跳槽的概率 array (index 0, index 1), index1 表示跳槽的概率

In [ ]:
model.predict_proba([[1,0,0,1,0,0,0,0,0,0,0,0, 0.5, 0.7, 4.0, 160, 3.0, 0, 0]])

In [ ]:
pred = model.predict(X)
pred_proba = model.predict_proba(X)
print(pred)
print(pred_proba)
(abs(pred-y)).sum() / len(y)

生成7:3的训练测试集并在训练集上训练模型model2

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=0)


In [ ]:
model2 = LogisticRegression(C=10000)
model2.fit(Xtrain, ytrain)
pred = model2.predict(Xtest)
metrics.accuracy_score(ytest, pred)


用metrics观察实际离职/未离职被预测成为离职/未离职的数目

In [ ]:
metrics.confusion_matrix(ytest, pred)

#        prediction
#
#
#actual
#
#
#

In [ ]:
print(metrics.classification_report(ytest, pred))
# 之所以有2个f1-score, 相当于 把跳槽看成正例， 把不跳槽看成正例，各算一次

10份的交叉验证Cross Validation

In [ ]:
#??? cv = 10 什么意思，validation, train 的set 比例在这10次train中，比列是什么样的？
print(cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=10))